In [9]:
import pandas as pd
dfk=pd.read_csv("kdramaf.csv")
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
dfk

,Name,Synopsis,Genre,Poster Path
0,Move to Heaven,Geu Roo is a young autistic man. He works for ...,"Life, Drama, Family",https://image.tmdb.org/t/p/original/n277gYwCQO...
1,Flower of Evil,Although Baek Hee Sung is hiding a dark secret...,"Thriller, Romance, Crime, Melodrama",https://m.media-amazon.com/images/M/MV5BMzY2NT...
2,Hospital Playlist,The stories of people going through their days...,"Friendship, Romance, Life, Medical",https://image.tmdb.org/t/p/original/wJJJZf9HKM...
3,Hospital Playlist 2,Everyday is extraordinary for five doctors and...,"Friendship, Romance, Life, Medical",https://i.mydramalist.com/dKY0d_4f.jpg
4,My Mister,Park Dong Hoon is a middle-aged engineer who i...,"Psychological, Life, Drama, Family",https://m.media-amazon.com/images/M/MV5BZjkzNm...
...,...,...,...,...
100,When the Camellia Blooms,"Woo Seo Ri, a violin prodigy at 17 who was abo...","Thriller, Comedy, Romance, Drama, Family",https://image.tmdb.org/t/p/original/9AUMDkXjCh...
101,Extracurricular,A high school becomes ground zero for a zombie...,"Psychological, Crime, Youth, Drama",https://i.pinimg.com/474x/43/a9/8a/43a98a86fbd...
102,Doctor Prisoner,The idealistic Choi In Ha has her work cut out...,"Thriller, Law, Crime, Medical",https://m.media-amazon.com/images/M/MV5BODIxMG...
103,Psychopath Diary,Jeon Nok Du lives on an island with his father...,"Thriller, Mystery, Comedy, Crime",https://m.media-amazon.com/images/M/MV5BYTA4Yj...


In [12]:
dfk['combined_features']=dfk['Genre']+dfk['Synopsis']
dfk['combined_features']


0      Life,  Drama,  Family Geu Roo is a young autis...
1      Thriller,  Romance,  Crime,  Melodrama Althoug...
2      Friendship,  Romance,  Life,  Medical The stor...
3      Friendship,  Romance,  Life,  Medical Everyday...
4      Psychological,  Life,  Drama,  Family Park Don...
                             ...                        
100    Thriller,  Comedy,  Romance,  Drama,  Family W...
101    Psychological, Crime, Youth, DramaA high schoo...
102    Thriller, Law, Crime, MedicalThe idealistic Ch...
103    Thriller, Mystery, Comedy, CrimeJeon Nok Du li...
104    Action, Thriller, CrimeTwo people have been fr...
Name: combined_features, Length: 105, dtype: object

In [14]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(dfk['combined_features'])

In [15]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [16]:
knn = NearestNeighbors(n_neighbors=10, metric='cosine')  # n_neighbors=10 means top-10 similar movies
knn.fit(cosine_sim)


NearestNeighbors(metric='cosine', n_neighbors=10)

In [18]:
def recommend_kdrama_by_title(kdrama_title):
    kd_index = dfk[dfk['Name'].str.lower() == kdrama_title.lower()].index[0]
    distance, indices = knn.kneighbors(cosine_sim[kd_index].reshape(1, -1), n_neighbors=10)
    recommended_kd_indices = indices.flatten()[1:]
    recommended_kd = dfk.iloc[recommended_kd_indices]
    return recommended_kd['Name']

kdrama = recommend_kdrama_by_title('Hospital Playlist')

# Print movie names without index
for i in kdrama:
    print(i)


Hospital Playlist 2
Dr. Romantic 2
Ghost Doctor
Signal
Dr. Romantic
Children of Nobody
The King of Pigs
Our Beloved Summer
Twenty-Five Twenty-One


In [19]:
import pickle
pickle.dump(dfk['combined_features'], open('kdrama_list.pkl', 'wb'))
pickle.dump(cosine_sim, open('similarity.pkl', 'wb'))
pickle.load(open('kdrama_list.pkl', 'rb'))

0      Life,  Drama,  Family Geu Roo is a young autis...
1      Thriller,  Romance,  Crime,  Melodrama Althoug...
2      Friendship,  Romance,  Life,  Medical The stor...
3      Friendship,  Romance,  Life,  Medical Everyday...
4      Psychological,  Life,  Drama,  Family Park Don...
                             ...                        
100    Thriller,  Comedy,  Romance,  Drama,  Family W...
101    Psychological, Crime, Youth, DramaA high schoo...
102    Thriller, Law, Crime, MedicalThe idealistic Ch...
103    Thriller, Mystery, Comedy, CrimeJeon Nok Du li...
104    Action, Thriller, CrimeTwo people have been fr...
Name: combined_features, Length: 105, dtype: object